# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [26]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [27]:
# Load API Credentials
with open('/Users/purvikansara/.secret/yelp_api.json','r') as f:
    login = json.load(f)
# with open('/Users/purvikansara/.secret/yelp_api.json') as f:
#     login = json.load(f)
# login.keys()

In [28]:
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [29]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['API Key'],timeout_s= 5.0)

### Define Search Terms and File Paths

In [30]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98122'
term = 'pizza'


In [31]:
## Specify fodler for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER,exist_ok=True)

In [32]:
location.split(',')[0]

'Seattle'

In [33]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [34]:
JSON_FILE

'Data/Seattle-pizza.json'

### Check if Json File exists and Create it if it doesn't

In [35]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Seattle-pizza.json not found. Saving empty list to file.


![png](Figures/question.png)

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [36]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term =term,location = location)

In [37]:
type(results)

dict

In [38]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [39]:
results['businesses'][0]

{'id': 'M9xzvwgK58T0w7wvXedvuQ',
 'alias': 'hot-mamas-pizza-seattle',
 'name': "Hot Mama's Pizza",
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qjUtdXO8aUPV9U-WwQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/hot-mamas-pizza-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
 'review_count': 949,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
 'rating': 4.0,
 'coordinates': {'latitude': 47.615379179632, 'longitude': -122.32323073437},
 'transactions': ['delivery'],
 'price': '$',
 'location': {'address1': '700 E Pine St',
  'address2': '',
  'address3': '',
  'city': 'Seattle',
  'zip_code': '98122',
  'country': 'US',
  'state': 'WA',
  'display_address': ['700 E Pine St', 'Seattle, WA 98122']},
 'phone': '+12063226444',
 'display_phone': '(206) 322-6444',
 'distance': 2080.359405276959}

In [40]:

pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,949,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",[delivery],$,"{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,2080.359405
1,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,336,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[delivery, pickup]",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796
2,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/AxKjCa...,False,https://www.yelp.com/biz/central-pizza-seattle...,201,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.598976, 'longitude': -122.294685}","[delivery, pickup]",$$,"{'address1': '2901 S Jackson St', 'address2': ...",+12066026333,(206) 602-6333,1571.867438
3,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113
4,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,96,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[delivery, pickup]",NaN,"{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393
5,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,127,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],$$,"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,8784.075722
6,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media1.fl.yelpcdn.com/bphoto/O9QKPW...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,36,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],NaN,"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1686.862176
7,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media3.fl.yelpcdn.com/bphoto/1EGE3Z...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,356,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],$$,"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011
8,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,995,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","[delivery, pickup]",$$,"{'address1': '1028 Madison St', 'address2': No...",+12065380040,(206) 538-0040,2271.307105
9,-FOAQv22SXtSBs7nptI3UA,serious-pie-downtown-seattle-2,Serious Pie Downtown,https://s3-media1.fl.yelpcdn.com/bphoto/IhsjHU...,False,https://www.yelp.com/biz/serious-pie-downtown-...,4455,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.61304626911818, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '2001 4th Ave', 'address2': None,...",+12068387388,(206) 838-7388,3348.309276


In [41]:
## How many results total?
results['total']

856

In [42]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [43]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [44]:
# Use math.ceil to round up for the total number of pages of results.
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

43

In [50]:
for i in tqdm_notebook( range(1,n_pages+1)):
   # The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location=location,
                                        term=term, 
                                        offset=n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
            
    #What to do if we get an error/exception.
    except Exception as e: # saving the error message so we can print it.
        print('[!] ERROR: ',e)

  0%|          | 0/43 [00:00<?, ?it/s]

![png](Figures/Poll_531.png)

## Open the Final JSON File with Pandas

In [51]:
df = pd.read_json(JSON_FILE)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,336,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[pickup, delivery]",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796
1,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/AxKjCa...,False,https://www.yelp.com/biz/central-pizza-seattle...,201,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.598976, 'longitude': -122.294685}","[pickup, delivery]",$$,"{'address1': '2901 S Jackson St', 'address2': ...",+12066026333,(206) 602-6333,1571.867438
2,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113
3,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,96,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[pickup, delivery]",NaN,"{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393
4,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,127,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],$$,"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,8784.075722


In [52]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle-pizza.csv.gz'

In [53]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [54]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 834,513 Bytes
CSV.GZ FILE: 119,679 Bytes
the csv.gz is 6.972927581279924 times smaller!


## Next Class: Processing the Results and Mapping 